In [9]:
from pathlib import Path
import pandas as pd
from tabulate import tabulate
from py import NS

dirs = NS.Dirs()
gf = NS.GlobalFiles()
ar = NS.AllResults()
rs = NS.RSSParams()
jn = NS.JupyNBReports()

CWD = Path.cwd()
CWD

PosixPath('/Users/mahdimir/Documents/Projects/Thesis-TeIAS')

# General Remarks
* Zero cost strategy returns in each month is equal to returns to winner(buy) portfolio minus the returns to losers(Sell) portfolio?
My ans. Yes
* All porfolios have long positions.

In [10]:
allr = pd.read_excel(CWD / f"{gf.allResults_Xl}{NS.xlsuf}")
# allr

In [11]:
#%# JT93 - Table 1
conf = {
        rs.nthWDayEachMonthForEval: [0],
        rs.mMonthSkip             : [0],
        rs.pDaysSkip              : [0, 7],
        rs.fromJMonth             : [138001],
        rs.toJMonth               : [139909],
        rs.evalMonths             : [3, 6, 9, 12],
        rs.holdingMonths          : [3, 6, 9, 12],
        rs.qCut                   : [10],
        ar.strategyType           : [ar.rebalanced],
        ar.binNo                  : [1, 10, 'w-l'], }

def filter_results_by_config(config_dict, all_results=allr):
    filtered = all_results.copy()
    for k, v in config_dict.items():
        filtered = filtered[filtered[k].isin(v)]
    return filtered

tbl1_0 = filter_results_by_config(conf)
# tbl1_0

In [12]:
def filter_table1_panel_a_or_b(tabl1, a_or_b):
    if a_or_b == 'a':
        skdays = 0
    elif a_or_b == 'b':
        skdays = 7
    else:
        raise 'a or b'
    pnl = tabl1[tabl1[ar.pDaysSkip].eq(skdays)]
    pnl = pnl[[ar.evalMonths, ar.holdingMonths, ar.binNo, ar.meanReturns,
               ar.meanTStat, ar.meanPVal, ]]
    pnl.loc[pnl[ar.binNo].eq(1), jn.portfo_type] = jn.short
    pnl.loc[pnl[jn.portfo_type].eq(jn.short), 'l0'] = 0
    pnl.loc[pnl[ar.binNo].eq(10), jn.portfo_type] = jn.long
    pnl.loc[pnl[jn.portfo_type].eq(jn.long), 'l0'] = 1
    pnl.loc[pnl[ar.binNo].eq('w-l'), jn.portfo_type] = jn.longMinusShort
    pnl.loc[pnl[jn.portfo_type].eq(jn.longMinusShort), 'l0'] = 2
    pnl = pnl.drop(columns = ar.binNo)
    return pnl

# Panel A
tbl1_pnl_a = filter_table1_panel_a_or_b(tbl1_0, a_or_b = 'a')
# tbl1_pnl_a

In [13]:
def tabulate_panel(panel):
    pnl_r = pd.pivot_table(panel,
                           columns = ar.holdingMonths,
                           values = ar.meanReturns,
                           index = [ar.evalMonths, jn.portfo_type, 'l0'])
    pnl_t = pd.pivot_table(panel,
                           columns = ar.holdingMonths,
                           values = ar.meanTStat,
                           index = [ar.evalMonths, jn.portfo_type, 'l0'])
    pnl_p = pd.pivot_table(panel,
                           columns = ar.holdingMonths,
                           values = ar.meanPVal,
                           index = [ar.evalMonths, jn.portfo_type, 'l0'], )

    pnl_r["l"] = 0
    pnl_t["l"] = 1
    pnl_p["l"] = 2

    pnl_r = pnl_r.reset_index()
    pnl_t = pnl_t.reset_index()
    pnl_p = pnl_p.reset_index()

    pnl = pnl_r.append(pnl_t)
    pnl = pnl.append(pnl_p)

    pnl = pnl.sort_values(by = [ar.evalMonths, 'l0', "l"])

    pnl.loc[pnl['l'].isin([0, 2]), [3, 6, 9, 12]] = pnl.loc[
        pnl['l'].isin([0, 2]), [3, 6, 9,
                                12]].applymap(lambda x: "{:.2f}%".format(100 * x))
    pnl.loc[pnl['l'].isin([1]), [3, 6, 9, 12]] = pnl.loc[
        pnl['l'].isin([1]), [3, 6, 9,
                             12]].applymap(lambda x: "{:.4f}".format(x))

    pnl["r-t-p"] = pnl[
        "l"].apply(lambda x: "Returns" if x == 0 else "TStat" if x == 1 else "PVal")

    pnl = pnl[[ar.evalMonths, jn.portfo_type, "r-t-p", 3, 6, 9, 12]]

    print(tabulate(pnl,
                   showindex = False,
                   headers = ["J", "", "K", "3", "6", "9", "12"]))

## Table 1 - Panel A
Without skipping a week between portfolio formation and holding period.

In [14]:
tabulate_panel(tbl1_pnl_a)
# #%% md
### Insignificant Results at 5% in Panel A

  J                  K        3        6        9       12
---  --------------  -------  -------  -------  ------  ------
  3  short           Returns  3.28%    3.28%    3.09%   2.95%
  3  short           TStat    5.4682   6.3820   6.0826  5.8611
  3  short           PVal     0.00%    0.00%    0.00%   0.00%
  3  long            Returns  1.73%    2.50%    3.48%   3.42%
  3  long            TStat    1.7052   2.9899   4.6122  4.7966
  3  long            PVal     8.95%    0.31%    0.00%   0.00%
  3  longMinusShort  Returns  -1.55%   -0.77%   0.39%   0.47%
  3  longMinusShort  TStat    -1.3905  -0.9879  0.6293  0.8721
  3  longMinusShort  PVal     16.57%   32.42%   52.98%  38.41%
  6  short           Returns  3.42%    3.18%    2.84%   2.87%
  6  short           TStat    6.3419   6.1130   5.6183  5.7952
  6  short           PVal     0.00%    0.00%    0.00%   0.00%
  6  long            Returns  2.30%    3.69%    3.59%   3.65%
  6  long            TStat    2.2069   3.3374   3.3870  3.6246
  6 

In [97]:
# tbl1_pnl_a_insig = tbl1_pnl_a[tbl1_pnl_a[ar.meanPVal] > 0.05]
# tbl1_pnl_a_insig

In [98]:
# print(tabulate(tbl1_pnl_a_insig, showindex = False, headers = 'keys'))

In [99]:
# % Table 1-Panel B
tbl1_pnl_b = filter_table1_panel_a_or_b(tbl1_0, a_or_b = 'b')
# tbl1_pnl_b

## Table 1 - Panel B

In [100]:
tabulate_panel(tbl1_pnl_b)

  J                  K        3       6       9       12
---  --------------  -------  ------  ------  ------  ------
  3  short           Returns  -3.70%  -3.26%  -3.13%  -3.03%
  3  short           TStat    6.2086  5.8521  5.7444  5.5360
  3  short           PVal     0.00%   0.00%   0.00%   0.00%
  3  long            Returns  1.65%   2.29%   3.12%   3.04%
  3  long            TStat    1.5763  2.5340  3.6520  4.1902
  3  long            PVal     11.64%  1.20%   0.03%   0.00%
  3  longMinusShort  Returns  5.35%   5.55%   6.26%   6.07%
  3  longMinusShort  TStat    4.1296  4.5187  5.1519  5.3395
  3  longMinusShort  PVal     0.01%   0.00%   0.00%   0.00%
  6  short           Returns  -3.34%  -3.09%  -2.89%  -2.90%
  6  short           TStat    5.7721  5.5632  5.3411  5.4210
  6  short           PVal     0.00%   0.00%   0.00%   0.00%
  6  long            Returns  2.33%   3.14%   3.15%   3.04%
  6  long            TStat    2.0540  2.9919  3.5239  3.5862
  6  long            PVal     4.11%

In [101]:
def are_all_zero_cost_portfolios_have_positive_return(table1=tbl1_0):
    all_p = table1.loc[table1[ar.binNo].eq('w-l'), ar.meanReturns].ge(0).all()
    return all_p

all_zero_p = are_all_zero_cost_portfolios_have_positive_return()

### Facts
1. All Zero-cost portfolios (winners minus losers) have postive returns.

   Just Like JT(93).

In [102]:
print(all_zero_p)

True


In [103]:
def are_all_zero_cost_portfolios_have_sig_returns(table1=tbl1_0):
    all_s = table1.loc[table1[ar.binNo].eq('w-l'), ar.meanPVal].lt(0.05).all()
    # print(all_p)
    return all_s

all_s_r = are_all_zero_cost_portfolios_have_sig_returns()

2. Are all zero-cost portfolios returns have significant returns at 5%?

    YES!

In [104]:
print(all_s_r)

True


3. How t-stats are calculated?

    For each stratgey mean returns are calculated by taking average of all months returns, then its t-stat is calculated by its own sample of returns for each month assuming that returns are normal and the null hypothesis is that the mean return is zero.

4. Which zero-cost strategy is the most successful?

In [122]:
def find_most_succ_zero_cost_strat_in_pnl(table1):
    idxmax = table1[table1[jn.portfo_type].eq(jn.longMinusShort)][
        ar.meanReturns].idxmax()
    # print(idxmax)
    ose = table1.loc[idxmax].drop('l0')
    return ose.to_frame()

#### In Panel A:
(9,6) Strategy

In [124]:
print(tabulate(find_most_succ_zero_cost_strat_in_pnl(tbl1_pnl_a)))

-------------  --------------------
evalMonths     9
holdingMonths  6
meanReturns    0.07973459231146847
meanTStat      4.540502364832509
meanPVal       9.12630888150539e-06
portfo_type    longMinusShort
-------------  --------------------


#### In Panel B:
(9,6) Strategy

In [126]:
print(tabulate(find_most_succ_zero_cost_strat_in_pnl(tbl1_pnl_b)))

-------------  ---------------------
evalMonths     9
holdingMonths  6
meanReturns    0.07489864171009505
meanTStat      4.723539012316343
meanPVal       4.070594496881118e-06
portfo_type    longMinusShort
-------------  ---------------------


* Why all short strategies have negative returns? This is in the contrary with JT(93) examination of the US market.
* Do JT(93) take short position on sell portfolio or long?

In [ ]:
mean()
